Several techniques are deployed to classify EMG
data such as artificial neural networks (ANN), Bayesian classifier (BC), fuzzy logic (FL), multilayer
perceptron (MLP), support vector machines (SVM), linear discriminant analysis (LDA), hidden Markov
models (HMM) and K-nearest neighbor (KNN). 

In [5]:
# Import packages
import pandas as pd
import glob
from sklearn.datasets import load_boston
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np
import math 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.kernel_ridge import KernelRidge

# Import libraries
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
#import seaborn as sns

#### Data Formatting
###### Include labels in individual csv folders
0: rest
1: fist
2: open palm

In [6]:
# I currently have 3 folders with 14 files each  (each file 1000 samples long)
# I want to merge them all into one file (they must be properly labeled first)


#""" I coudl make multiple paths to the original csv files and loop through so that I do not have to manually edit the path 
#I can do the same for the 'label' input """
#
#path = '/home/cybathlon/catkin_ws/src/bagFiles/rest' # all these must be labeled 0
#all_files = glob.glob(path + "/*.csv") #all files ending in csv
#
#onefile = []
#
#for filename in all_files: #read through all csv files and append them to each other
#    df = pd.read_csv(filename, index_col=None, header=0)
#    onefile.append(df)
#    
##TODO: modify the input of 'label' column
#concatdf = pd.concat(onefile, axis=0, ignore_index=True) #concatenate all files into a full dataframe
#concatdf['label'] = '0' #add column label of zeros
#concatdf.to_csv(path + '0full.csv', index=False)#save df to csv


###### Join all files into one for analysis

In [7]:
# now that I have 3 csv files labeled, I can concatenate them to have one big file with all my data
# reuse code above

#path = '/home/cybathlon/catkin_ws/src/bagFiles'
#all_files = glob.glob(path + "/*.csv") #all files ending in csv
#
#eachfile = []
#
#for filename in all_files: #read through all csv files and append them to each other
#    df = pd.read_csv(filename, index_col=None, header=0)
#    eachfile.append(df)
#
#fulldf = pd.concat(eachfile, axis=0, ignore_index=True)
#fulldf.to_csv(path + 'full_csv.csv', index=False) #save df to csv

# Read Data

##### Is the data correctly set and split?

In [8]:
# Import data set
# Import data set
df = pd.read_csv("/home/cybathlon/catkin_ws/src/bagFiles/bagFilesfull_csv.csv")
#df = pd.read_csv(r"C:\Users\Paula\Desktop\9weekproject\gitwork\EMGMyoRosClassification\bagFiles\full_csv.csv")
x = df[["EMG0", "EMG1", "EMG2", "EMG3", "EMG4", "EMG5", "EMG6", "EMG7"]].values
y = df["label"].values

In [9]:
# Split data into train/test set using 80% train 20% test
x_train , x_test , y_train , y_test = train_test_split(x , y , shuffle=True, test_size=0.2, random_state=42)

# Standardizing

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
scaler = StandardScaler() # define scaler

In [12]:
print(scaler.fit(x_train)) # fit scaler on the training dataset
print(scaler.mean_)
print(scaler.transform(x))

StandardScaler(copy=True, with_mean=True, with_std=True)
[ 75.95016204  86.99583333  92.65518519 198.70185185 306.37851852
 101.19851852 149.56972222  74.53810185]
[[ 0.30063111  2.38126717  6.15343404 ...  0.49970343  0.7708874
   0.21755642]
 [ 0.27336278  2.02662084  5.29969901 ...  0.52686013  0.74485056
   0.20139552]
 [ 0.09611865  1.79018995  4.19932941 ...  0.71695705  0.66023085
   0.10443011]
 ...
 [-0.80373619 -0.99631695 -0.67644624 ... -1.03465023 -0.84990558
  -0.86522405]
 [-0.81737036 -0.97942903 -0.71439002 ... -1.03465023 -0.84990558
  -0.88138495]
 [-0.79010203 -0.87810151 -0.60055868 ... -1.03465023 -0.85641479
  -0.88138495]]


In [13]:
# transform both datasets 
x_train = scaler.transform(x_train) #x_train_scaled = scaler.fit(x_train)
x_test = scaler.transform(x_test)
print(x_test)

[[ 0.35516777  0.4391563  -0.27803656 ...  0.39107662  3.64795766
   2.69017452]
 [ 1.66404754 -0.16880884 -0.29700845 ... -0.27426257  0.15902178
   0.45996996]
 [-0.80373619 -1.18208408 -1.41634994 ... -0.6544564  -0.8694332
  -0.88138495]
 ...
 [-0.32654044  0.30405294 -0.31598034 ... -0.84455332 -0.84339637
  -0.70361502]
 [ 2.27758493 -0.0505934  -0.27803656 ...  0.21455806  1.43482669
   2.07606022]
 [-0.72193121 -1.03009279 -0.94205269 ... -0.72234816 -0.83688716
  -0.86522405]]


# Base Line Model (Without Optimization)

In [14]:
clf = svm.SVC(kernel = 'poly', cache_size=7000) #create svm classifier with polynomial kernel (maybe try with rbg)
clf.fit(x_train,y_train) #train the model using training set

SVC(C=1.0, cache_size=7000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [15]:
y_pred_train = clf.predict(x_train)
y_pred_test = clf.predict(x_test)

In [16]:
print(classification_report(y_test, y_pred_test))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3566
          1       1.00      1.00      1.00      3597
          2       1.00      1.00      1.00      3637

avg / total       1.00      1.00      1.00     10800



In [17]:
#Model Accuracy: how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_test))

('Accuracy:', 0.9980555555555556)


In [18]:
#Model Precision: what percentage of positive tuples are labeled as such?
#Model Recall: what percentage of positive tuples are labeled as such?
print("Precision:", metrics.precision_score(y_test, y_pred_test, average = 'micro'))
print("Recall:", metrics.recall_score(y_test, y_pred_test, average = 'micro'))

('Precision:', 0.9980555555555556)
('Recall:', 0.9980555555555556)


# Parameter Tuning

In [19]:
#Parameter Tuning:
#defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['poly']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, n_jobs = -1) 
  
#fitting the model for grid search with scaled dataset
grid.fit(x_train, y_train) 

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] kernel=poly, C=0.1, gamma=1 .....................................
[CV] kernel=poly, C=0.1, gamma=1 .....................................
[CV] kernel=poly, C=0.1, gamma=1 .....................................
[CV] kernel=poly, C=0.1, gamma=0.1 ...................................
[CV] kernel=poly, C=0.1, gamma=0.1 ...................................
[CV] kernel=poly, C=0.1, gamma=0.1 ...................................
[CV]  kernel=poly, C=0.1, gamma=1, score=0.999583333333, total=   0.5s
[CV] kernel=poly, C=0.1, gamma=0.01 ..................................
[CV]  kernel=poly, C=0.1, gamma=1, score=0.998888811723, total=   0.5s
[CV] kernel=poly, C=0.1, gamma=0.01 ..................................
[CV]  kernel=poly, C=0.1, gamma=1, score=0.999583362267, total=   0.5s
[CV] kernel=poly, C=0.1, gamma=0.01 ..................................
[CV]  kernel=poly, C=0.1, gamma=0.1, score=0.992361111111, total=   5.8s
[CV] kernel=po

[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:  2.0min


[CV]  kernel=poly, C=1, gamma=0.01, score=0.91221612612, total=  36.7s
[CV] kernel=poly, C=1, gamma=0.001 ...................................
[CV]  kernel=poly, C=0.1, gamma=0.0001, score=0.33414346226, total=  48.8s
[CV] kernel=poly, C=1, gamma=0.0001 ..................................
[CV]  kernel=poly, C=0.1, gamma=0.0001, score=0.334097222222, total=  49.5s
[CV] kernel=poly, C=1, gamma=0.0001 ..................................
[CV]  kernel=poly, C=0.1, gamma=0.0001, score=0.33412042503, total=  50.6s
[CV] kernel=poly, C=1, gamma=0.0001 ..................................
[CV]  kernel=poly, C=1, gamma=0.001, score=0.33414346226, total=  53.0s
[CV] kernel=poly, C=10, gamma=1 ......................................
[CV] . kernel=poly, C=10, gamma=1, score=0.999236164155, total=   0.4s
[CV] kernel=poly, C=10, gamma=1 ......................................
[CV] . kernel=poly, C=10, gamma=1, score=0.999513888889, total=   0.4s
[CV] kernel=poly, C=10, gamma=1 ...............................

[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  6.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'kernel': ['poly'], 'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [20]:
# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

{'kernel': 'poly', 'C': 0.1, 'gamma': 1}
SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [21]:
# define model
model = grid.best_estimator_
model.fit(x_train, y_train) # model includes scaling 

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
# testing the model
y_pred = grid.predict(x_test)

In [23]:
# performance analysis of the model
class_rep = classification_report(y_test, y_pred)
conf_mat = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(class_rep)
print('\n\nConfusion Matrix : \n')
print(conf_mat)
print('\n\n Accuracy Score : ', accuracy)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3566
          1       1.00      1.00      1.00      3597
          2       1.00      1.00      1.00      3637

avg / total       1.00      1.00      1.00     10800



Confusion Matrix : 

[[3566    0    0]
 [   2 3595    0]
 [   0    4 3633]]
('\n\n Accuracy Score : ', 0.9994444444444445)


In [25]:
#model persistence
import pickle
# save the model
pickle.dump(model, open("svmClassifier1030.pkl", "wb"),protocol =2)
# save the scaler
pickle.dump(scaler, open('scaler1030.pkl', 'wb'), protocol =2)

In [ ]:
# visualization of the predicted class vs actual class (0 based indexing)
plt.figure(figsize = (15, 10))
classes = [i for i in range(1, 4)]
svm_df = pd.DataFrame(conf_mat, index = classes, columns = classes)
sns.heatmap(svm_df, annot = True)
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')

In [ ]:
# Create models
knn = KNeighborsClassifier()
svc1 = SVC(kernel = 'linear', gamma ='scale')
svc2 = SVC(kernel = 'poly', gamma ='scale')
svc3 = SVC(kernel = 'rbf', gamma ='scale')

model_list = [knn, svc1, svc2, svc3]
model_name_list = ["KNN", "linear SVC", "poly SVC", "rbf SVC"]


In [ ]:
# Create meshgrid
x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.02), np.arange(x2_min, x2_max, 0.02))

fig = plt.figure(figsize = (15,15))

Features were extracted from the samples by segmenting the signal into 250 ms intervals corresponding to 256 samples in each segment. A single feature was calculated from each segment and the segment window was incremented by 125 ms (128 samples) for the next feature. This scheme ensured that a control command could be generated within 250 ms from the instant the user's intention was given. Three kinds of features were extracted from each segment namely EMG rms value, AR model coefficients and WL.

(source: https://biomedical-engineering-online.biomedcentral.com/articles/10.1186/1475-925X-9-41)

The Myo Armband already computes the rms value (not sure how)

AR models are constructed using a recursive filter. This filter predicts the current value based on the previous output values of the filter. The current value y(t) can be computed as:
see math in link above

We used the AR model coefficients as the features with a model order of four, which is adequate for modelling EMG signals [42], thus generating four features for each channel of sEMG.

The third kind of extracted feature was the waveform length, which provided a measure of the waveform complexity in each segment. The waveform length l can be mathematically represented as:

We used four channels of sEMG data, which therefore provided 24 features per segment. As regards to classification, the LibSVM tool [43] was used in the Matlab environment. LibSVM has an implementation for multi class SVM using one-versus-one strategy and provides a choice of four basic kernels namely linear, polynomial, radial basis function (RBF) and sigmoid. As discussed in [44, 45], RBF is in general a reasonable first choice as it maps the samples nonlinearly and has few numbers of hyperparameters reducing the complexity of model selection. 